In [7]:
import sqlite3
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

conn = sqlite3.connect('database.sqlite')

def main(): 
    stats, X,y = getData()
    print("Done")
    # clf = DecisionTreeClassifier(criterion="entropy", random_state=1234)
    # score= cross_val_score(clf, X.values, y, cv=5)


def getData():
    with sqlite3.connect('database.sqlite') as con:
        countries = pd.read_sql_query("SELECT * from Country", con)
        match_data = pd.read_sql("SELECT * from Match", con)
        leagues = pd.read_sql_query("SELECT * from League", con)
        teams = pd.read_sql_query("SELECT * from Team", con)
        player = pd.read_sql_query("SELECT * from Player",con)
        player_attributes = pd.read_sql("SELECT * from Player_Attributes",con)
        sequence = pd.read_sql("SELECT * from sqlite_sequence",con)
        team_attributes = pd.read_sql_query("SELECT * from Team_Attributes",con)
   
    rows = ["home_player_1", "home_player_2", "home_player_3", "home_player_4", "home_player_5", "home_player_6", "home_player_7", 
        "home_player_8", "home_player_9", "home_player_10", "home_player_11", "away_player_1",
        "away_player_2", "away_player_3", "away_player_4", "away_player_5", "away_player_6",
        "away_player_7", "away_player_8", "away_player_9", "away_player_10", "away_player_11", "home_team_api_id", "away_team_api_id"]
    match_data.dropna(subset = rows, inplace = True)

    y= []
    stats= pd.DataFrame()
    (n,d)= match_data.shape

    for match_indx in range(10):
        match = match_data.iloc[match_indx]
        label = get_match_label(match)
        team_attr = get_team_stats(match, team_attributes)
        player_stats = get_fifa_stats(match_data.iloc[match_indx], player_attributes)
        # and team_attr.notna().all(axis=None)
        if(label!=0 and team_attr is not None):
            y.append(label)
            toAppend= pd.concat([player_stats, team_attr.reindex(player_stats.index)], axis=1)
            stats= stats.append(toAppend)
            # print(stats.shape)
        

    # X = stats.drop(columns=['match_api_id'])
    X = stats
    y = pd.DataFrame(y)

    X.to_pickle("data/X_players_score.pkl")
    y.to_pickle("data/y_players_score.pkl")
    return stats, X, y

def get_team_stats(match, team_attributes):
    if(match.home_team_api_id is np.nan or match.away_team_api_id is np.nan): 
        return None 
    date = match["date"]
    team_stats = pd.DataFrame()
    team_attrs = ['date', 'buildUpPlaySpeed', 'buildUpPlayPassing', 'chanceCreationPassing', 'chanceCreationCrossing', 'chanceCreationShooting', 
                'defencePressure', 'defenceAggression', 'defenceTeamWidth']   
    # team_attrs = ['date', 'buildUpPlaySpeed']   
    
    home_team_stats = team_attributes[team_attributes.team_api_id == match['home_team_api_id']]
    home_team_stats = home_team_stats[team_attrs]
    current_home_team_stats = home_team_stats[home_team_stats.date < date].sort_values(by = 'date', ascending = False)[:1]
    team_stats = current_home_team_stats[team_attrs].drop(columns=['date']).add_prefix("home_")
    
    away_team_stats = team_attributes[team_attributes.team_api_id == match['away_team_api_id']]
    away_team_stats = away_team_stats[team_attrs]
    current_away_team_stats = away_team_stats[away_team_stats.date < date].sort_values(by = 'date', ascending = False)[:1]
    team_stats = pd.concat([team_stats,current_home_team_stats[team_attrs].drop(columns=['date']).add_prefix("away_").reindex()], axis=1)
    if(not team_stats.empty):
        # print(team_stats)
        return team_stats 
    return None


# Code based on the work done by Pavan Raj on Kaggle
def get_match_label(match):
    ''' Derives a label for a given match. '''
    
    #Define variables
    home_goals = match['home_team_goal']
    away_goals = match['away_team_goal']

    #Identify match label  
    if home_goals > away_goals:
        return 1
        # label.loc[0,'label'] = "Win"
    if home_goals == away_goals:
        return 0
        # label.loc[0,'label'] = "Draw"
    if home_goals < away_goals:
        return -1
        # label.loc[0,'label'] = "Defeat"

    return 0        

# Code based on the work done by Pavan Raj on Kaggle
def get_fifa_stats(match, player_stats):
    ''' Aggregates fifa stats for a given match. '''    
    
    #Define variables
    match_id =  match.match_api_id
    date = match['date']
    players = ['home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
               "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
               "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
               "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
               "away_player_10", "away_player_11"]
    player_stats_new = pd.DataFrame()
    names = []
    
    #Loop through all players
    for player in players:   
            
        #Get player ID
        player_id = match[player]
        
        #Get player stats 
        stats = player_stats[player_stats.player_api_id == player_id]
            
        #Identify current stats       
        current_stats = stats[stats.date < date].sort_values(by = 'date', ascending = False)[:1]
        
        if np.isnan(player_id) == True:
            overall_rating = pd.Series(0)
        else:
            current_stats.reset_index(inplace = True, drop = True)
            overall_rating = pd.Series(current_stats.loc[0, "overall_rating"])

        #Rename stat
        name = "{}_overall_rating".format(player)
        names.append(name)
            
        #Aggregate stats
        player_stats_new = pd.concat([player_stats_new, overall_rating], axis = 1)
    
    player_stats_new.columns = names        
    player_stats_new['match_api_id'] = match_id

    player_stats_new.reset_index(inplace = True, drop = True)
    
    #Return player stats    
    return player_stats_new    

main()

Done


In [8]:
print(stats.head())

NameError: name 'stats' is not defined

In [6]:
    with sqlite3.connect('database.sqlite') as con:
        countries = pd.read_sql_query("SELECT * from Country", con)
        match_data = pd.read_sql("SELECT * from Match", con)
        leagues = pd.read_sql_query("SELECT * from League", con)
        teams = pd.read_sql_query("SELECT * from Team", con)
        player = pd.read_sql_query("SELECT * from Player",con)
        player_attributes = pd.read_sql("SELECT * from Player_Attributes",con)
        sequence = pd.read_sql("SELECT * from sqlite_sequence",con)
        team_attributes = pd.read_sql_query("SELECT * from Team_Attributes",con)
    team_attrs = ['date', 'buildUpPlaySpeed', 'buildUpPlayPassing', 'chanceCreationPassing', 'chanceCreationCrossing', 'chanceCreationShooting', 
                'defencePressure', 'defenceAggression', 'defenceTeamWidth']   
    print(team_attributes[team_attrs].describe())

buildUpPlaySpeed  buildUpPlayPassing  chanceCreationPassing  \
count       1458.000000         1458.000000            1458.000000   
mean          52.462277           48.490398              52.165295   
std           11.545869           10.896101              10.360793   
min           20.000000           20.000000              21.000000   
25%           45.000000           40.000000              46.000000   
50%           52.000000           50.000000              52.000000   
75%           62.000000           55.000000              59.000000   
max           80.000000           80.000000              80.000000   

       chanceCreationCrossing  chanceCreationShooting  defencePressure  \
count             1458.000000             1458.000000      1458.000000   
mean                53.731824               53.969136        46.017147   
std                 11.086796               10.327566        10.227225   
min                 20.000000               22.000000        23.000000   
25%   

In [9]:
try:
    X = pd.read_pickle("data/X_players_score.pkl")
    y = pd.read_pickle("data/y_players_score.pkl")
except:
    print("error")
    X,y = getData()
print(X.head())

home_player_1_overall_rating  home_player_2_overall_rating  \
0                          64.0                          65.0   
0                          64.0                          65.0   
0                          71.0                          72.0   
0                          67.0                          62.0   
0                          67.0                          65.0   

   home_player_3_overall_rating  home_player_4_overall_rating  \
0                          64.0                          67.0   
0                          64.0                          67.0   
0                          73.0                          70.0   
0                          64.0                          63.0   
0                          67.0                          63.0   

   home_player_5_overall_rating  home_player_6_overall_rating  \
0                          63.0                          65.0   
0                          64.0                          65.0   
0                         